In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
##from pandas.plotting import parallel_coordinates
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
from sklearn import tree
plt.style.use("seaborn-dark")

In [ ]:
df = pd.read_csv('weatherAUS.csv')

<h3> Cantidad de lugares distintos de la base de datos </h3>

In [ ]:
len(df['Location'].unique())

<h4> Hacemos una copia del la base de datos para cambiar los valores de 'yes' y 'no' por valores booleanos en la copia del DF </h4>

In [ ]:
data = df.copy()

In [ ]:
data['RainToday'] = (data['RainToday'] == 'Yes')*1
data['RainTomorrow'] = (data['RainTomorrow'] == 'Yes')*1

<h4>Cambiamos las abreviaciones de las direcciones del viento en las respectivas columnas </h4>

In [ ]:
Ab_WD = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S', 'SSW','SW','WSW','W','WNW','NW','NNW']
WD = [0,22.5,45,67.5,90,112.5,135,157.5,180,202.5,225,247.5,270,292.5,315,337.5]
Wind_Dir = pd.DataFrame()
Wind_Dir['Ab_WD']=Ab_WD
Wind_Dir['WD']=WD

In [ ]:
#for i in Wind_Dir.index:
 #   datc.loc[datc['WindGustDir']== Wind_Dir.iloc[i][0],'WindGustDir']= Wind_Dir.iloc[i][1]

In [ ]:
#for i in Wind_Dir.index:
 #   datc.loc[datc['WindDir9am']== Wind_Dir.iloc[i][0],'WindDir9am']= Wind_Dir.iloc[i][1]

In [ ]:
#for i in Wind_Dir.index:
 #   datc.loc[datc['WindDir3pm']== Wind_Dir.iloc[i][0],'WindDir3pm']= Wind_Dir.iloc[i][1]

In [ ]:
Col_WindDir=['WindGustDir','WindDir9am','WindDir3pm']
for j in Col_WindDir:
    for i in Wind_Dir.index:
        data.loc[data[j] == Wind_Dir.iloc[i][0], j]= Wind_Dir.iloc[i][1]

<h4> Ahora veamos la cantidad de valores nulos que hay en la base de datos</h4>

<h4>Cantidad de calores nulos </h4>

In [ ]:
data.info()

<h4>Calculemos los coeficientes de correlación de la base de datos para darnos una idea de que columnas pueden ser de utlidad y cuales no. </h4>

In [ ]:
cor_data = data.corr()
cor_data

<h4> Visualicemos esto con un gráfico </h4>

In [ ]:
plt.figure(figsize=(20,12))
sns.heatmap(cor_data,annot=True)


<h4>Cantidad de calores nulos en porcentaje </h4>

In [ ]:
(data.isnull().sum()*100)/len(data.index)

<h4>Dado que queremos un modelo que prediga si lloverá o no en una o varias de las ciudades de la base de datos, lo que haremos será optar por predecir con un modelo de árbol de desiciones (clasificador) </h4>

<h4>Puesto que queremos predecir especificamente en cada ciudad, como primer acercamiento separaremos los datos de cada ciudad en la base de datos y la limpiaremos para proceder a crear el modelo. </h4>

Para esto creamos un diccionario que contenga las bases de datos de cada lugar


In [ ]:
Loc = data['Location'].unique()
df_dict_Loc = {elem : pd.DataFrame() for elem in Loc}
for key in df_dict_Loc.keys():
    df_dict_Loc[key] = data[:][data.Location == key]
df_dict_Loc

In [ ]:
df_dict_Loc

Ahora procedemos a limpiar cada base de datos. 

Lo primero sera ver cuantos (porcentaje) valores nulos hay en cada base de datos.

In [ ]:
null_val_perc = {elem : pd.Series(dtype='float64') for elem in Loc}
for key in df_dict_Loc.keys():
    null_val_perc[key] = (df_dict_Loc[key].isnull().sum()*100)/len(df_dict_Loc[key].index)

In [ ]:
null_val_perc

Veamos las columnas con mas valores nulos y borremos las columnas que tengan todos sus valores nulos

In [ ]:
for key in null_val_perc.keys():
    for col in df_dict_Loc[key].columns:
        if null_val_perc[key][col] == 100.0:
            df_dict_Loc[key]=df_dict_Loc[key].drop([col],axis=1)
            null_val_perc[key]=null_val_perc[key].drop([col])

In [ ]:
df_dict_Loc

In [ ]:
null_val_perc

Dado que todavia hay columnas con valores nulos, en esta ocasión optaremos por rellenar los valores con la media o moda dependiendo del tipo de datos.

Columnas que se rellenan con la moda.

In [ ]:
Col_Mode =['WindGustDir','WindDir9am','WindDir3pm','Cloud9am','Cloud3pm']
for key in df_dict_Loc.keys():
    for col in Col_Mode:
        if col in df_dict_Loc[key].columns:
            df_dict_Loc[key][col]=df_dict_Loc[key][col].fillna(df_dict_Loc[key][col].mode()[0])

Columnas que se rellenan con la media.

In [ ]:
for key in df_dict_Loc.keys():
    df_dict_Loc[key].fillna(df_dict_Loc[key].mean(), inplace=True)

Ahora no hay base de datos con valores nulos. Corroboremos esto.

In [ ]:
null_val_perc = {elem : pd.Series(dtype='float64') for elem in Loc}
for key in df_dict_Loc.keys():
    null_val_perc[key] = (df_dict_Loc[key].isnull().sum()*100)/len(df_dict_Loc[key].index)

In [ ]:
null_val_perc

In [ ]:
Ahora veamos los coeficientes de correlación y decidamos con cuales características qeudarnos 
para el modelo.

In [ ]:
coef_corr = {elem : pd.DataFrame() for elem in Loc}
for key in df_dict_Loc.keys():
    coef_corr[key]= df_dict_Loc[key].corr()

In [ ]:
coef_corr

In [ ]:
for keys in df_dict_Loc.keys():
    plt.figure(figsize=(20,12))
    sns.heatmap(coef_corr[key],annot=True)

Para el modelo elegiremos las columnas con coeficientes cuyos valores absolutos son
mayores o iguales a .1 

Coeficientes positivos

In [ ]:
coef_corr_p = {elem : pd.DataFrame() for elem in Loc}
for key in df_dict_Loc.keys():
    coef_corr_p[key]= coef_corr[key][coef_corr[key]['RainTomorrow']>= .1]
    coef_corr_p[key]=coef_corr_p[key].drop(['RainTomorrow'],axis=0)

In [ ]:
coef_corr_p

Coeficientes negativos

In [ ]:
coef_corr_n = {elem : pd.DataFrame() for elem in df_dict_Loc.keys()}
for key in df_dict_Loc.keys():
    coef_corr_n[key]= coef_corr[key][coef_corr[key]['RainTomorrow']<= -.1]

In [ ]:
coef_corr_n

Ahora tomemos esas características

In [ ]:
total_len = {elem : list() for elem in Loc}
for key in df_dict_Loc.keys():
    total_len[key].append(len(coef_corr_p[key].index) +len(coef_corr_n[key].index))

In [ ]:
total_len['Albury'][0]

In [ ]:
features = {elem : list() for elem in Loc}
for key in df_dict_Loc.keys():
    for i in range(0,total_len[key][0]):
        if i<=(len(coef_corr_p[key])-1):
            features[key].append(coef_corr_p[key].index[i])
        elif i >= len(coef_corr_p[key]):
            features[key].append(coef_corr_n[key].index[i-(len(coef_corr_p[key])+1)])

In [ ]:
features

In [ ]:
Pasemos a crear los modelos

In [ ]:
X = {elem : pd.DataFrame() for elem in Loc}
for key in df_dict_Loc.keys():
    X[key]= df_dict_Loc[key][features[key]].copy()

In [ ]:
X

In [ ]:
y = {elem : pd.DataFrame() for elem in Loc}
for key in df_dict_Loc.keys():
    y[key]= df_dict_Loc[key][['RainTomorrow']].copy()

In [ ]:
y

Ya tenemos todo para hacer el modelo, podemos elegir cualquier "llave " del
diccionario y ver que tan bueno es el modelo

In [ ]:
key='Canberra'
X_train, X_test, y_train, y_test = train_test_split(X[key], y[key], test_size=0.33, random_state=324)

In [ ]:
val_met=pd.DataFrame(columns=['node','accuracy'],index=range(22))
for i in range(3,25):
    Rain_Tomorrow = DecisionTreeClassifier(max_leaf_nodes=i, random_state=0)
    Rain_Tomorrow.fit(X_train, y_train)
    predictions = Rain_Tomorrow.predict(X_test)
    j=i-3
    val_met.iloc[j][0] = i 
    val_met.iloc[j][1]=(accuracy_score(y_true = y_test, y_pred = predictions))
    print(i,' ',accuracy_score(y_true = y_test, y_pred = predictions))